In [66]:
!pip install openpyxl

In [1]:
import pandas as pd

# Load the Excel file
file_path = 'interopSurvey.xlsx'
df = pd.read_excel(file_path)

# Inspect the first few rows of the DataFrame to understand its structure
df.head()


,Q52,Q71,Q1,Q2,Q5,Q67,Q67_6_TEXT,Q9,Q11,Q20,...,Q61,Q62,Q62_3_TEXT,Q63,Q63_3_TEXT,Q78,Q64,Q65,Q65_3_TEXT,Q66
0,Do you consent to participate in this study?,Do you interact regularly with a machine learn...,How many years have you worked on Machine Lear...,How many years have you worked on Software Eng...,What is the size of your organization?,"In the context of machine learning, what deplo...","In the context of machine learning, what deplo...",Approximately how many pre-trained model packa...,Approximately how many pre-trained model packa...,Which naming convention do you prefer when reu...,...,Do you use ONNX as part of your model developm...,"Other than ONNX, do you use other interoperabi...","Other than ONNX, do you use other interoperabi...",For what purpose do you use interoperability t...,For what purpose do you use interoperability t...,If you use ONNX for framework-to-framework con...,Do you ever deploy directly from a deep learni...,Do you commonly encounter problems while worki...,Do you commonly encounter problems while worki...,"If you encounter such problems, how do you add..."
1,NaN,NaN,NaN,NaN,NaN,Desktop application,NaN,NaN,NaN,AnswerDisplayOrder,...,"Yes, I use ONNX",Other (please specify),NaN,NaN,NaN,NaN,NaN,AnswerDisplayOrder,NaN,NaN
2,"Yes, I consent","Yes, I have and/or currently interact regularl...",3 - 5 years,1 - 2 years,Small (1 - 50 employees),"Web application,Desktop application,IoT / embe...",NaN,10 - 20,0,AnswerDisplayOrder,...,"Yes, I use ONNX",NaN,NaN,"Model conversion for deployment (e.g., convert...",NaN,NaN,I have never deployed from PyTorch or TensorFl...,AnswerDisplayOrder,NaN,Typically when I encounter crashes I try a dif...
3,"Yes, I consent","Yes, I have and/or currently interact regularl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,AnswerDisplayOrder,...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Yes, I consent","Yes, I have and/or currently interact regularl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Yes, I use ONNX",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Total number of rows where the first two columns include yes
yes_rows = df[df['Q52'].str.contains('yes', case=False, na=False) & df['Q71'].str.contains('yes', case=False, na=False)]
print(f"Total number of rows where the first two columns include yes: {len(yes_rows)}")

Total number of rows where the first two columns include yes: 91


In [4]:
# check how many rows are not null in column 21
df[df.columns[21]].notnull().sum()


91

In [5]:
# Columns T to AE in Excel correspond to indices 19 to 30 in Python (0-based indexing)
columns_of_interest = df.columns[19:31]  # Select columns T to AE

# Count the number of rows where columns T to AE are not all 'n/a'
# We will treat both 'n/a' and NaN as equivalent for this purpose
count_non_na_rows = df[columns_of_interest].replace('n/a', pd.NA).dropna(how='all').shape[0]

count_non_na_rows


93

In [6]:
# Columns T to AE in Excel correspond to indices 19 to 30 in Python (0-based indexing)
columns_of_interest = df.columns[20:31]  # Select columns T to AE

# Count the number of rows where columns T to AE are not all 'n/a'
# We will treat both 'n/a' and NaN as equivalent for this purpose
count_non_na_rows = df[columns_of_interest].replace('n/a', pd.NA).dropna(how='all').shape[0]

count_non_na_rows


92

In [7]:
# Columns T to AE in Excel correspond to indices 19 to 30 in Python (0-based indexing)
columns_of_interest = df.columns[19:31]  # Select columns T to AE

# Count the number of rows where columns T to AE are not all 'n/a'
# We will treat both 'n/a' and NaN as equivalent for this purpose
count_non_na_rows = df[columns_of_interest].replace('n/a', pd.NA).dropna(how='all').shape[0]

count_non_na_rows


# Remove the rows where columns_of_interest are all null values and save this in a new Excel file
df = df[~df[columns_of_interest].replace('n/a', pd.NA).isnull().all(axis=1)]

output_file_path = 'interopSurvey.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Output saved to: {output_file_path}")



Output saved to: interopSurvey.xlsx


In [8]:
# Open interopSurvey.xlsx
df = pd.read_excel('interopSurvey.xlsx')

# For each column, count the number of rows for each unique value. This is like a sets of unique values in each column and their counts
unique_values = {col: df[col].value_counts() for col in df.columns}


In [9]:
# count the categories in Q67. Some of them has this format "category1, category2, category3"
# We will split them and count the number of each category
categories = df['Q67'].str.split(',').explode().str.strip().value_counts()

categories



Web application                                                                           59
Cloud and data center                                                                     52
Desktop application                                                                       19
IoT / embedded systems                                                                    14
Mobile                                                                                    14
Other (please specify)                                                                     4
In the context of machine learning                                                         1
what deployment environments do you focus on? (check all that apply) - Selected Choice     1
Name: Q67, dtype: int64

In [10]:
# Count how many df['Q65'] are not null
Q65_sum = df['Q65'].notnull().sum()
print(f"Total number of non-null values in column Q65: {Q65_sum}")

# Define a placeholder that does not appear in your data
placeholder = "eg"

# Replace 'e.g.,' with the placeholder, split, explode, and then replace the placeholder back
ONNX_bug_categories = (
    df['Q65']
    .str.replace('e.g.,', placeholder, regex=False)  # Temporarily replace 'e.g.,' with a placeholder
    .str.split(',')  # Split on commas
    .explode()  # Convert lists to rows
    .str.strip()  # Strip whitespace
    .str.replace(placeholder, 'e.g.,', regex=False)  # Replace the placeholder back to 'e.g.,'
    .value_counts()  # Count occurrences
)

# Calculate the percentage of each category (int)
ONNX_bug_categories_percentage = round(ONNX_bug_categories / Q65_sum * 100)

# Combine ONNX_bug_categories and ONNX_bug_categories_percentage into a DataFrame
ONNX_bug_summary = pd.DataFrame({'Count': ONNX_bug_categories, 'Percentage': ONNX_bug_categories_percentage})



# Print     df['Q65_3_TEXT'] value which is not null
print(f"Number of non-null values in column Q65_3_TEXT: {df['Q65_3_TEXT'].notnull().sum()}")
print(df['Q65_3_TEXT'].replace('n/a', pd.NA).dropna().unique())


ONNX_bug_summary



Total number of non-null values in column Q65: 54
Number of non-null values in column Q65_3_TEXT: 6
['Do you commonly encounter problems while working with ONNX models? - Other (please specify) - Text'
 'individual implementation issues for ONNX runtimes'
 'Maybe not exactly what you’re looking for, but an onnx dep in a python package has a slower release cadence on macOS '
 'Not much ONNX experience.' "ONNX doesn't support Fourier layers."
 'Speed or difficulty deploying ONNX to user machines due to driver problems']


,Count,Percentage
I do not commonly encounter problems with ONNX models,22,41.0
"Crashes (e.g., model does not convert to ONNX)",19,35.0
"Performance differences (e.g., the accuracy of the ONNX model does not match the original model)",19,35.0
Other (please specify),6,11.0
AnswerDisplayOrder,3,6.0
Do you commonly encounter problems while working with ONNX models? - Selected Choice,1,2.0


In [11]:
# print out the first row in df.columns[21]
print(f"Column 21: {df[df.columns[21]].iloc[0]}")

Column 21: Do you use ONNX as part of your model development and delpoyment process?


In [12]:
# Same analysis for Q63
Q63_sum = df['Q63'].notnull().sum()
print(f"Total number of non-null values in column Q63: {Q63_sum}")

placeholder = "eg"

usage_categories = (
    df['Q63']
    .str.replace('e.g.,', placeholder, regex=False)  # Temporarily replace 'e.g.,' with a placeholder
    .str.split(',')  # Split on commas
    .explode()  # Convert lists to rows
    .str.strip()  # Strip whitespace
    .value_counts()  # Count occurrences
)

usage_categories_percentage = round(usage_categories / Q63_sum * 100)

usage_summary = pd.DataFrame({'Count': usage_categories, 'Percentage': usage_categories_percentage})

usage_summary

Total number of non-null values in column Q63: 54


,Count,Percentage
Model conversion for deployment (eg converting to ONNX for deployment using ONNXRuntime or TensorRT),37,69.0
Framework-to-framework Model Conversion (eg converting models from TensorFlow to PyTorch),28,52.0
Other (please specify),3,6.0
For what purpose do you use interoperability tools? (check all that apply) - Selected Choice,1,2.0
